In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)   # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [2]:
x[0].shape      # (1, 28, 28)
x[1].shape      # (1, 28, 28)

(1, 28, 28)

In [3]:
x[0, 0]     # 또는 x[0][0]

array([[5.17262495e-01, 1.73825774e-01, 5.15414652e-01, 4.72881305e-01,
        8.81924124e-01, 6.55351712e-02, 2.23549157e-01, 3.53446645e-01,
        2.58769174e-01, 9.18896168e-01, 5.60683149e-01, 8.96574558e-01,
        1.19867229e-01, 5.33387162e-01, 1.44708090e-01, 9.31532183e-01,
        5.63857065e-01, 9.95055757e-02, 9.48489443e-02, 3.49659887e-01,
        2.19932776e-01, 3.47215468e-01, 4.58924869e-01, 3.56315460e-01,
        7.89526387e-01, 3.43861573e-01, 8.46685200e-01, 1.48370345e-01],
       [4.69890656e-01, 6.03847316e-01, 6.68096615e-01, 1.19846353e-01,
        9.89164609e-01, 1.77398329e-01, 5.59737292e-01, 5.02350854e-01,
        2.90788483e-01, 2.79758296e-01, 2.16343028e-01, 9.50926909e-01,
        6.41795150e-01, 6.71877350e-01, 9.54661366e-01, 2.58426844e-01,
        3.43933982e-01, 9.79428995e-01, 2.02916063e-01, 3.21423186e-02,
        4.77036866e-01, 7.93484661e-01, 1.09538005e-01, 4.99368790e-01,
        1.37689439e-01, 2.89384040e-01, 6.46294451e-01, 2.15523

In [4]:
%cd /content/drive/MyDrive/밑바닥부터 시작하는 딥러닝/ch07

/content/drive/MyDrive/밑바닥부터 시작하는 딥러닝/ch07


In [5]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)     # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)   # (9, 75)

x2 = np.random.rand(10, 3, 7, 7)    # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)   # (90, 75)

(9, 75)
(90, 75)


In [6]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T    # 필터 전개
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

In [7]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape

        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_W) / self.stride)

        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 최댓값 (2)
        out = np.max(col, axis=1)

        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [8]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num':30, 'filter_size':5,
                             'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]

        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],
                                        self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],
                                        self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers: 
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads